In [1]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
import json
import pandas as pd
import numpy as np

In [84]:
train_file = open('./round1_train_0907.json', encoding='utf-8')
test_file = open('./round1_test_0907.json', encoding='utf-8')

train_data = json.load(train_file)
test_data = json.load(test_file)

In [72]:
test_data[0].keys()

dict_keys(['id', 'text', 'annotations'])

In [73]:
test_data[0]

{'id': 1050,
 'text': '"橄榄，又名青果、白榄，为橄榄科植物橄榄的果实，产广东、广西、福建等地。宋朝大文学家苏东坡称之为“青子”。早在唐宋之间，橄榄已广泛地被采入药用。现代研究证实，橄榄的果实中含有蛋白质、脂肪、碳水化合物以及钙.磷、铁等。祖国医学认为橄榄味甘酸，性平，能够清肺，利咽，生津：解毒，主治咽喉肿痛，烦渴，咳嗽咯血以及细菌性痢疾、癫痫等，还能解除河豚毒以及酒毒。用新鲜橄榄3枚，白萝卜数片，水煎服，可以用于治疗咽喉肿痛。用橄榄10枚，去核，水煎汤，频频服用，可以治疗饮酒中毒昏闷不适。若肠风下血，可用橄榄烧灰存性，每次6克，用米汤汁调服。将橄榄炒研为末，用猪油调和，外敷，可以治疗口唇干裂生疮。用鲜橄榄20枚，冰糖50克，水炖服，可以用于小儿百日咳的治疗。若妇女妊娠呕吐不止，可将鲜橄榄适量捣烂，用水煎服。"',
 'annotations': [{'Q': '', 'A': '橄榄的果实中含有蛋白质、脂肪、碳水化合物以及钙.磷、铁等。'},
  {'Q': '', 'A': '清肺，利咽，生津：解毒'},
  {'Q': '', 'A': '青子'},
  {'Q': '', 'A': '主治咽喉肿痛，烦渴，咳嗽咯血以及细菌性痢疾、癫痫等，还能解除河豚毒以及酒毒。'}]}

In [74]:
len(test_data)

1399

In [85]:
all_data_list = []
for i in range(len(train_data)):
    for j in range(len(train_data[i]['annotations'])):
        dict_cur = {}
        dict_cur['id'] = train_data[i]['id']
        dict_cur['text'] = train_data[i]['text']
        dict_cur['question'] = train_data[i]['annotations'][j]['Q']
        dict_cur['answer'] = train_data[i]['annotations'][j]['A']
        all_data_list.append(dict_cur)

In [86]:
id_list = []
text_list = []
question_list = []
answer_list = []
for i in range(len(all_data_list)):
    id_list.append(all_data_list[i]['id'])
    text_list.append(all_data_list[i]['text'])
    question_list.append(all_data_list[i]['question'])
    answer_list.append(all_data_list[i]['answer'])

In [88]:
train_df = pd.DataFrame(columns = ["id", "text", "question", "answer"])
train_df['id'] = id_list
train_df['text'] = text_list
train_df['question'] = question_list
train_df['answer'] = answer_list
print(train_df.shape)
train_df.to_csv('./train.csv', index=False)

(18478, 4)


In [89]:
all_test_data_list = []
for i in range(len(test_data)):
    for j in range(len(test_data[i]['annotations'])):
        dict_cur = {}
        dict_cur['id'] = test_data[i]['id']
        dict_cur['text'] = test_data[i]['text']
        dict_cur['question'] = test_data[i]['annotations'][j]['Q']
        dict_cur['answer'] = test_data[i]['annotations'][j]['A']
        all_test_data_list.append(dict_cur)

In [90]:
test_id_list = []
test_text_list = []
test_question_list = []
test_answer_list = []
for i in range(len(all_test_data_list)):
    test_id_list.append(all_test_data_list[i]['id'])
    test_text_list.append(all_test_data_list[i]['text'])
    test_question_list.append(all_test_data_list[i]['question'])
    test_answer_list.append(all_test_data_list[i]['answer'])

In [91]:
test_df = pd.DataFrame(columns = ["id", "text", "question", "answer"])
test_df['id'] = test_id_list
test_df['text'] = test_text_list
test_df['question'] = test_question_list
test_df['answer'] = test_answer_list
print(test_df.shape)
test_df.to_csv('./test.csv', index=False)

(4373, 4)


In [78]:
def text_segmentate(text, maxlen, seps='\n', strips=None):
    """将文本按照标点符号划分为若干个短句
    """
    text = text.strip().strip(strips)
    if seps and len(text) > maxlen:
        pieces = text.split(seps[0])
        text, texts = '', []
        for i, p in enumerate(pieces):
            if text and p and len(text) + len(p) > maxlen - 1:
                texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
                text = ''
            if i + 1 == len(pieces):
                text = text + p
            else:
                text = text + p + seps[0]
        if text:
            texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
        return texts
    else:
        return [text]

In [79]:
# 筛选数据
max_p_len = 512

seps, strips = u'\n。！？!?；;，, ', u'；;，, '
data = []
                    
for idx in range(train_df.shape[0]):
    if train_df['answer'][idx]:
        for t in text_segmentate(train_df['text'][idx], max_p_len-2, seps, strips):
            if train_df['answer'][idx] in t:
                data.append((t, train_df['question'][idx], train_df['answer'][idx]))

len(data)

18454

In [80]:
data[0]

('"胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习惯。有症状的胆囊结石仍以胆囊切除术为较安全有效的疗法，此外，尚可采用体外震波碎石。胆管结石宜采用以手术为主的综合治疗。胆石症的家庭治疗可采用以下方法：\\n（1）一般治疗    预防和治疗肠道寄生虫病和肠道感染，以降低胆石症的发病率。胆绞痛发作期应禁食脂肪等食物，采用高碳水化合物流质饮食；缓解期应忌食富含胆固醇的食物如脑、肝、肾、蛋黄等。\\n（2）增进胆汁排泄    可选用50%硫酸镁10~15毫升，餐后口服，每日3次；胆盐每次口服0.5~1克，每日3次；去氢胆酸0.25克，每日3次，餐后服用。\\n（3）消除胆绞痛    轻者可卧床休息，右上腹热敷，用硝酸甘油酯0.6毫克，每3~4小时一次，含于舌下；或阿托品0.5毫克，每3~4小时肌肉注射一次。重者应住院治疗。\\n（4）排石疗法以中药治疗为主，若右上腹疼痛有间歇期，无明显发热及黄疸，苔薄白，脉弦，属气滞者，用生大黄6克、木香9克、枳壳9克、金钱草30克、川楝子9克、黄苓9克，水煎服。右上腹痛为持续性，且阵发性加剧，有明显发热及黄疸，舌红苔黄',
 '什么类型的胆囊结石可不作治疗？',
 '无症状胆囊结')

In [82]:
random_order = list(range(len(data)))
# np.random.shuffle(random_order)
# json.dump(random_order, open('../random_order.json', 'w'), indent=4)

# 划分valid
train_data_after = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
valid_data_after = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]

In [83]:
train_data_after[0]

('"胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习惯。有症状的胆囊结石仍以胆囊切除术为较安全有效的疗法，此外，尚可采用体外震波碎石。胆管结石宜采用以手术为主的综合治疗。胆石症的家庭治疗可采用以下方法：\\n（1）一般治疗    预防和治疗肠道寄生虫病和肠道感染，以降低胆石症的发病率。胆绞痛发作期应禁食脂肪等食物，采用高碳水化合物流质饮食；缓解期应忌食富含胆固醇的食物如脑、肝、肾、蛋黄等。\\n（2）增进胆汁排泄    可选用50%硫酸镁10~15毫升，餐后口服，每日3次；胆盐每次口服0.5~1克，每日3次；去氢胆酸0.25克，每日3次，餐后服用。\\n（3）消除胆绞痛    轻者可卧床休息，右上腹热敷，用硝酸甘油酯0.6毫克，每3~4小时一次，含于舌下；或阿托品0.5毫克，每3~4小时肌肉注射一次。重者应住院治疗。\\n（4）排石疗法以中药治疗为主，若右上腹疼痛有间歇期，无明显发热及黄疸，苔薄白，脉弦，属气滞者，用生大黄6克、木香9克、枳壳9克、金钱草30克、川楝子9克、黄苓9克，水煎服。右上腹痛为持续性，且阵发性加剧，有明显发热及黄疸，舌红苔黄',
 '胆石症的治疗应注意什么？',
 '应区别不同情况分别处理')

In [92]:
for idx, (p,q,a) in enumerate(train_data_after):
    print(idx, p, q ,a)
    break

0 "胆石症的治疗应区别不同情况分别处理，无症状胆囊结石可不作治疗，但应定期观察并注意良好的饮食习惯。有症状的胆囊结石仍以胆囊切除术为较安全有效的疗法，此外，尚可采用体外震波碎石。胆管结石宜采用以手术为主的综合治疗。胆石症的家庭治疗可采用以下方法：\n（1）一般治疗    预防和治疗肠道寄生虫病和肠道感染，以降低胆石症的发病率。胆绞痛发作期应禁食脂肪等食物，采用高碳水化合物流质饮食；缓解期应忌食富含胆固醇的食物如脑、肝、肾、蛋黄等。\n（2）增进胆汁排泄    可选用50%硫酸镁10~15毫升，餐后口服，每日3次；胆盐每次口服0.5~1克，每日3次；去氢胆酸0.25克，每日3次，餐后服用。\n（3）消除胆绞痛    轻者可卧床休息，右上腹热敷，用硝酸甘油酯0.6毫克，每3~4小时一次，含于舌下；或阿托品0.5毫克，每3~4小时肌肉注射一次。重者应住院治疗。\n（4）排石疗法以中药治疗为主，若右上腹疼痛有间歇期，无明显发热及黄疸，苔薄白，脉弦，属气滞者，用生大黄6克、木香9克、枳壳9克、金钱草30克、川楝子9克、黄苓9克，水煎服。右上腹痛为持续性，且阵发性加剧，有明显发热及黄疸，舌红苔黄 胆石症的治疗应注意什么？ 应区别不同情况分别处理
